In [20]:
import pandas as pd
import difflib
dataset =  pd.read_csv('preprints_final_abstracts.csv')


In [76]:
def show_diff(text, n_text):
    """
    Based upon http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    
    seqm = difflib.SequenceMatcher(None, text, n_text)
    ratio = float(1.0 - seqm.ratio())
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=green>" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<strike><font color=red>" + seqm.a[a0:a1] + "</font></strike>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<strike><font color=red>"+ seqm.a[a0:a1] + "</font></strike> ""<font color=green>" + seqm.b[b0:b1] + "</font>")
        else:
            raise (RuntimeError, "unexpected opcode")
    return ratio, ''.join(output)

In [82]:
from IPython.core.display import display, HTML

for index, row in dataset.iterrows():
    abs1 = row["v1 Preprint abstract"]
    abs2 = row["Paper abstract"]
    try:
        change_ratio, out = show_diff(abs1,abs2)
        if  change_ratio> 0.2:
            print ("Scored abstract change:", row["Scored abstract change"])
            print ("Computed change ratio:", change_ratio)
#            print (" ")
#            print (abs1)
#            print (" ")
#            print (abs2)
            print (" ")
            display(HTML(out))
            print (" ")
            break
            
    except Exception as e:
            continue

Scored abstract change: No change or minor: change in wording, if any, does not change main conclusion(s)
Computed change ratio: 0.4515710205141522
 
